In [1]:
!pip install gym==0.23.1 --quiet
!pip install tensorflow --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.2/626.2 kB 22.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [1]:
#Library for environments
import gym
from gym.envs.registration import register
from gym.envs.toy_text.frozen_lake import generate_random_map

#Librairies to represent the output
from IPython.display import clear_output
import time
import matplotlib.pyplot as plt

#Essential libraries for computation
import numpy as np
import random
import tensorflow as tf
from collections import deque

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
random_map = generate_random_map(size=10, p=0.3)
env = gym.make("FrozenLake-v1", desc=random_map)
env.reset()
plt.imshow(env.render('rgb-array'))

In [3]:
# Get GPU device name
if tf.test.gpu_device_name():
    print('\nDefault GPU Device:', tf.test.gpu_device_name())
else:
    print('\nNo GPU device found')


Default GPU Device: /device:GPU:0


In [2]:
class DQNAgent:
    """
    Deep Q-Network agent for the FrozenLake environment.

    The agent uses a neural network to approximate the Q-function and
    implements epsilon-greedy exploration strategy with decay.
    """
    def __init__(self, state_size, action_size):
        # Set device before other initializations
        self.device_name = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device_name}")

        # Environment parameters
        self.state_size = state_size
        self.action_size = action_size

        # Learning parameters
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Initial exploration rate
        self.epsilon_min = 0.1  # Minimum exploration rate
        self.epsilon_decay = 0.995  # Decay rate for exploration
        self.learning_rate = 0.001  # Learning rate for the optimizer

        # Memory for experience replay
        self.memory = deque(maxlen=2000)
        self.batch_size = 32

        # Create the neural network model
        self.model = self._build_model()
        self.target_model = self._build_model()
        self.update_target_model()  # Initialize target model to be same as model

        # Optimizer
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)

    def _build_model(self):
        """
        Neural Network to approximate Q-value function:
        * Input: state
        * Output: Q-values for each action
        """
        model = nn.Sequential(
            nn.Linear(self.state_size, 24),
            nn.ReLU(),
            nn.Linear(24, 24),
            nn.ReLU(),
            nn.Linear(24, self.action_size)
        ).to(self.device_name)
        
        return model

    def update_target_model(self):
        """Update the target model with the weights of the model"""
        self.target_model.load_state_dict(self.model.state_dict())

    def remember(self, state, action, reward, next_state, done):
        """Store experience in memory for replay"""
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        """
        Choose action using epsilon-greedy policy
        """
        if random.random() <= self.epsilon:
            return random.randrange(self.action_size)

        state = torch.FloatTensor(state).unsqueeze(0).to(self.device_name)
        with torch.no_grad():
            q_values = self.model(state)
        return torch.argmax(q_values[0]).item()

    def replay(self):
        """
        Train the network using experience replay
        """
        if len(self.memory) < self.batch_size:
            return

        # Sample random experiences from memory
        minibatch = random.sample(self.memory, self.batch_size)

        states = torch.FloatTensor([x[0] for x in minibatch]).to(self.device_name)
        next_states = torch.FloatTensor([x[3] for x in minibatch]).to(self.device_name)
        actions = torch.LongTensor([x[1] for x in minibatch]).to(self.device_name)
        rewards = torch.FloatTensor([x[2] for x in minibatch]).to(self.device_name)
        dones = torch.BoolTensor([x[4] for x in minibatch]).to(self.device_name)

        # Predict Q-values for current and next states
        current_q_values = self.model(states)
        next_q_values = self.target_model(next_states)

        # Create training targets
        targets = current_q_values.clone()

        for i in range(self.batch_size):
            if dones[i]:
                targets[i, actions[i]] = rewards[i]
            else:
                targets[i, actions[i]] = rewards[i] + self.gamma * torch.max(next_q_values[i])

        # Compute loss
        loss = nn.MSELoss()(current_q_values, targets)

        # Optimize the model
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # Decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def convert_state(self, state):
        """Convert state number to one-hot encoding"""
        state_one_hot = np.zeros(self.state_size)
        state_one_hot[state] = 1
        return state_one_hot

# Training loop helper function
def train_dqn_agent(env, agent, episodes=1000):
    """
    Train the DQN agent on the environment
    """
    scores = []
    for episode in range(episodes):
        # Reset environment
        state, _ = env.reset()
        state = agent.convert_state(state)
        score = 0
        done = False

        while not done:
            # Choose and take action
            action = agent.act(state)
            next_state, reward, done, _, _ = env.step(action)

            # Convert next_state to one-hot and store experience
            next_state = agent.convert_state(next_state)
            agent.remember(state, action, reward, next_state, done)

            # Move to next state
            state = next_state
            score += reward

            # Train the network
            agent.replay()

        scores.append(score)

        # Print progress
        if (episode + 1) % 100 == 0:
            avg_score = np.mean(scores[-100:])
            print(f"Episode: {episode + 1}, Average Score: {avg_score:.2f}, Epsilon: {agent.epsilon:.2f}")

    return scores

# Helper function to create and initialize the agent
def create_agent(env):
    """
    Create and initialize a DQN agent for the given environment
    """
    state_size = env.observation_space.n  # Number of states
    action_size = env.action_space.n      # Number of actions
    agent = DQNAgent(state_size, action_size)
    return agent

In [3]:
# Initialize the environment and agent
random_map = generate_random_map(size=4, p=0.1)
env = gym.make("FrozenLake-v1", desc=random_map, is_slippery=True)
agent = create_agent(env)

# Training parameters
n_episodes = 1000
max_steps = 100  # Maximum steps per episode
training_history = {
    'scores': [],
    'avg_scores': [],
    'epsilons': [],
    'steps': []
}

Using device: cpu


In [4]:
# Training loop
for episode in range(n_episodes):
    state = env.reset()  # Just get the state
    state = agent.convert_state(state)
    score = 0
    done = False
    steps = 0

    for step in range(max_steps):
        # Choose and take action
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)  # Updated step unpacking

        # Convert next_state and store experience
        next_state = agent.convert_state(next_state)
        agent.remember(state, action, reward, next_state, done)

        # Move to next state
        state = next_state
        score += reward
        steps += 1

        # Train the network
        agent.replay()

        if done:
            break

    # Store training history
    training_history['scores'].append(score)
    training_history['epsilons'].append(agent.epsilon)
    training_history['steps'].append(steps)

    # Calculate average score over last 100 episodes
    avg_score = np.mean(training_history['scores'][-100:])
    training_history['avg_scores'].append(avg_score)

    # Print progress
    if (episode + 1) % 50 == 0:
        print(f"Episode: {episode + 1}/{n_episodes}")
        print(f"Score: {score:.2f}")
        print(f"Average Score (last 100): {avg_score:.2f}")
        print(f"Epsilon: {agent.epsilon:.3f}")
        print(f"Steps: {steps}")
        print("-" * 40)
        # Optional: render the environment to see the agent's behavior
        if episode % 100 == 0:
            env.render()

# Print final training results
print("\nTraining completed!")
print(f"Final average score: {training_history['avg_scores'][-1]:.2f}")
print(f"Final epsilon: {agent.epsilon:.3f}")

C:\Users\mztan\AppData\Local\Temp\ipykernel_29600\3991647148.py:82: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:281.)
  states = torch.FloatTensor([x[0] for x in minibatch]).to(self.device_name)


Episode: 50/1000
Score: 0.00
Average Score (last 100): 0.00
Epsilon: 0.363
Steps: 13
----------------------------------------
Episode: 100/1000
Score: 0.00
Average Score (last 100): 0.01
Epsilon: 0.100
Steps: 1
----------------------------------------
Episode: 150/1000
Score: 0.00
Average Score (last 100): 0.02
Epsilon: 0.100
Steps: 74
----------------------------------------
Episode: 200/1000
Score: 0.00
Average Score (last 100): 0.04
Epsilon: 0.100
Steps: 12
----------------------------------------
Episode: 250/1000
Score: 0.00
Average Score (last 100): 0.05
Epsilon: 0.100
Steps: 6
----------------------------------------
Episode: 300/1000
Score: 0.00
Average Score (last 100): 0.04
Epsilon: 0.100
Steps: 7
----------------------------------------
Episode: 350/1000
Score: 0.00
Average Score (last 100): 0.04
Epsilon: 0.100
Steps: 6
----------------------------------------
Episode: 400/1000
Score: 0.00
Average Score (last 100): 0.03
Epsilon: 0.100
Steps: 15
------------------------------